## Problem 1

In [2]:
import numpy as np
npts = 700
x = np.zeros(npts, dtype = np.float64)
for i in range(0,npts):
    x[i] = i
i = np.zeros(npts, dtype = np.int32)
for j in range(0,npts):
    i[j] = j
    
#watch out for for loops for trivial things. use numpy functions

The first 300 elements of x and i correspond to one halo's main progenitor, the next 50 a second trunk, the final 350 a third halo. Use numpy's built in mmap functionality to create a memory-mapped array of x and i on disk. Use your mmap object to fill in the blanks. In particular, do *not* load the entire array into ram to solve this problem. (You will need to make use of the *offset* keyword argument).

In [6]:
from tempfile import mkdtemp
import os.path as path
import numpy as np

tempfile_x = path.join(mkdtemp(), 'newfile_x.dat')
tempfile_i = path.join(mkdtemp(), 'newfile_i.dat')

offset_64 = 8 #64 bits, 8 bytes
offset_32 = 4 #32 bits, 4 bytes
#itemsize

mmap_x = np.memmap(tempfile_x, dtype='float64', mode='w+', shape=(1,npts))
mmap_x[:] = x[:]
mmap_y = np.memmap(tempfile_x, dtype='float64', mode='w+', offset = npts*offset_64, shape=(1,10))
mmap_y[:] = [0,1,2,3,4,5,6,7,8,9]

mmap_i = np.memmap(tempfile_i, dtype='int32', mode='w+', shape=(1,npts))
mmap_i[:] = i[:]

# print mmap_i





halo1_x = np.memmap(tempfile_x, dtype='float64', mode='r', shape=(1,300))
halo2_x = np.memmap(tempfile_x, dtype='float64', mode='r', offset = offset_64 * 300, shape=(1,50))
halo3_x = np.memmap(tempfile_x, dtype='float64', mode='r', offset = offset_64 * 350, shape=(1,360))

halo1_i = np.memmap(tempfile_i, dtype='int32', mode='r', shape=(1,300))
halo2_i = np.memmap(tempfile_i, dtype='int32', mode='r', offset = offset_32 * 300, shape=(1,50))
halo3_i = np.memmap(tempfile_i, dtype='int32', mode='r', offset = offset_32 * 350, shape=(1,350))

print halo3_x
# 
# halo1_x = elements of x of halo 1 read directly from your memory-mapped array on disk
# halo2_i = elements of i of halo 1 read directly from your memory-mapped array on disk

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0. 

## Problem 2

Use the python `multiprocessing` module to parallelize the following loop using all the cores available on your laptop. Benchmark your solution's runtime against the naive for loop, and also verify that you get the same answers in both cases. Make a plot showing how the speedup changes as a function of the number of threads. 

In [110]:
#Without multiprocessing

def expensive_calculation(seed):
    np.random.seed(seed)
    npts = 1e6
    x = np.random.uniform(-seed, seed, npts)
    return np.std(x)

num_iterations = 100

def no_multiprocessing():
    result = np.zeros(num_iterations)
    for i in range(num_iterations):
        result[i] = expensive_calculation(i)
    print result.shape
    return result

%time no_multiprocessing()


(100,)
CPU times: user 1.55 s, sys: 105 ms, total: 1.65 s
Wall time: 1.66 s


array([  0.        ,   0.57712386,   1.1565306 ,   1.73187954,
         2.31150049,   2.88528458,   3.46326808,   4.03991677,
         4.61869117,   5.1966822 ,   5.77462407,   6.35218527,
         6.92995853,   7.50360269,   8.08768231,   8.65650335,
         9.2330127 ,   9.81982101,  10.38519286,  10.96782556,
        11.54706617,  12.12779699,  12.71234068,  13.28026089,
        13.86060628,  14.43021811,  15.00633877,  15.58854204,
        16.17023345,  16.74902707,  17.31969731,  17.90034951,
        18.47243013,  19.05988842,  19.62493696,  20.19570678,
        20.80935613,  21.36342534,  21.92838869,  22.51558047,
        23.10502353,  23.67761255,  24.2416425 ,  24.82570991,
        25.42225262,  25.99598734,  26.55666513,  27.13400886,
        27.70576623,  28.29038877,  28.87014742,  29.43144828,
        30.03549569,  30.5973715 ,  31.16619662,  31.74646953,
        32.33592631,  32.89863375,  33.49117019,  34.08561123,
        34.64966909,  35.20612025,  35.79713006,  36.36

In [117]:
import multiprocessing as mp
def expensive_calculation(seed, output):
    np.random.seed(seed)
    npts = 1e6

    x = np.random.uniform(-seed, seed, npts)
    output.put(np.std(x))

def with_multiprocessing():
    output = mp.Queue()

    processes = [mp.Process(target=expensive_calculation, args=(seed, output)) for seed in range(0, num_iterations)]

    # Run processes
    for p in processes:
        p.start()

    # Exit the completed processes
    for p in processes:
        p.join()

    # Get process results from the output queue
    results = np.array([output.get() for p in processes])
t = %time with_multiprocessing()
print t


CPU times: user 74.5 ms, sys: 231 ms, total: 305 ms
Wall time: 1.13 s
None


## Problem 3

* Read chapter 3 in the PySpark book
* You may also find the following lecture useful: https://www.youtube.com/watch?v=ph374fJqFPE 